In [ ]:
# mounting g-drive
from google.colab import drive
drive.mount('/content/drive')

In [ ]:
import requests

In [ ]:
import matplotlib.pyplot as plt

In [ ]:
# loading prompts from google docs
doc_id = "1VJHTKDMPapyLbACAIOWvzRdrUhV70y03Mbyr_y--AN0"
url = f"https://docs.google.com/document/d/{doc_id}/export?format=txt"
response = requests.get(url)

# decoding with utf-8-sig to remove BOM
text = response.content.decode("utf-8-sig")
prompts = [line.strip() for line in text.split("\n") if line.strip()]


In [ ]:
# total number of prompts and first few prompts
print(f"Loaded {len(prompts)} prompts")
print("First 3 prompts:", prompts[:3])

In [ ]:
prompts

In [ ]:
import os

# setting output directory for Generated images from the prompts
output_dir = "/content/drive/MyDrive/Colab Notebooks/J Application Workstation/Dreamslayer/final_outputs"
os.makedirs(output_dir, exist_ok=True)


In [ ]:
# importing modelling libraries
import torch
from PIL import Image

In [ ]:
### model

In [ ]:
from diffusers import AutoPipelineForText2Image

In [ ]:
device = "cuda" if torch.cuda.is_available() else "cpu"

# pipeline for generation
pipe = AutoPipelineForText2Image.from_pretrained("stabilityai/sd-turbo",
                                                 dtype=torch.float16,
                                                 variant="fp16")
# moving to the device
pipe.to(device)

# enabling faster generation
pipe.enable_attention_slicing()


In [ ]:
# demo
demo_prompt = prompts[0]
print(demo_prompt)
demo = pipe(prompt=demo_prompt, num_inference_steps=4, guidance_scale=0.1).images[0]


In [ ]:
plt.imshow(demo)

In [ ]:
import time

In [ ]:
# from PIL import ImageEnhance

In [ ]:
## batch processing - 49 images from 49 prompts

from tqdm import tqdm

total_time = 0

for idx, prompt in enumerate(tqdm(prompts, desc="Generating images")):
    try:
        ind_time_start = time.time()
        image = pipe(prompt, height=512, width=512, num_inference_steps=4, guidance_scale=0.1).images[0]
        ind_time_ends = time.time()
        time_taken = ind_time_ends-ind_time_start
        print('Time taken to generate this image: ', ind_time_ends-ind_time_start)
        #sharpened = ImageEnhance.Sharpness(image).enhance(3.0)
        filename = os.path.join(output_dir, f"{idx+1:04d}.png")
        image.save(filename)
        print(f"Saved {filename}")
        total_time += time_taken
    except Exception as e:
        print(f"Failed for prompt {idx+1}: {prompt}\n{e}")

print(f"Total time taken for {len(prompts)} images: {total_time/60} minutes")

In [ ]:
print(total_time/60, "minutes")

In [ ]:
counter = 0
for i in os.listdir(output_dir):
  print(i + "----" +  prompts[counter])
  img = plt.imread(f"{output_dir}/{i}")
  plt.imshow(img)
  plt.show()
  counter += 1
  print("\n\n")